- **Conectando com Google Drive**

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

# **1 - Carregar bibliotecas**

In [ ]:
from skimage import io, color                                                    #Módulo para manipulação de imagens
from os import listdir
from os.path import isfile, join
import random
from skimage.feature import local_binary_pattern                                 #Módulo para cálculo do LBP
import numpy as np                                                               #Módulo para a manipulação de arrays
import pandas as pd                                                              #Módulo para a manipulação de dataframes
import matplotlib.pyplot as plt                                                  #Módulo para a manipulação de gráficos
import seaborn as sns                                                            #Módulo para a manipulação de gráficos
import os, glob                                                                  #Módulos para manipular estruturas de diretório
import cv2                                                                       #Módulo para manipulação de imagens

# **2 - Lista de Espécies**

O banco de dados possui imagens de casca (Ritidoma) 16 espécies de árvores comercialmente exploradas na Amazônia brasileira:

In [ ]:
pd.read_excel('./Class.xlsx', index_col=0)

,Specie
n,
1,Apuleia leiocarpa
2,Astronium lecointei
3,Bagassa guianensis
4,Bowdichia nitida
5,Cedrela odorata
6,Dipteryx odorata
7,Erisma uncinatum
8,Goupia glabra
9,Hymenolobium heterocarpum


# **3 - Contagem de Espécies e Imagens**

In [ ]:
base = glob.glob('./Amazon_Bark/*/*')
especies = glob.glob('./Amazon_Bark/*')

# Contagem do número de imagens no banco de dados completo
def contagem(caminho):
  cont_imgtotal = 0

  for filename in base:
    if filename.lower().endswith('.jpg'):
      cont_imgtotal += 1
  return(cont_imgtotal)

# Contagem do número espécies do banco de dados
def cont_esp(caminho):
  cont_sp = 0

  for filename in especies:
    pastas = os.path.join('.', filename)
    #print(pastas)
    if os.path.isdir(pastas):
      cont_sp += 1
  return(cont_sp)

In [ ]:
total_base = contagem(base)
total_sp = cont_esp(especies)
print(f'O total de imagens da base de dados é: {total_base}')
print(f'O total de espécies é: {total_sp}')

O total de imagens da base de dados é: 2803
O total de espécies é: 16


# **4 - Local Binary Pattern (LBP)**

O LBP (Local Binary Pattern) é um descritor de textura muito utilizado em visão computacional.

A biblioteca skimage já possui a função **"local_binary_pattern"** disponível no módulo **"feature()"**.

Para aplicar o descritor lbp em imagem devemos seguir os seguintes passos:
1. Importar as bibliotecas e funções que serão utilizadas;
2. Importar a/as imagem/imagens, que serão submetidas ao algoritmo;
3. Definir o tipo (configuração) do lbp que será utilizado:
    * Method: 'padrão' (escala de cinzas e não invariante a rotação), 'ror' (escala de cinza e invariante a rotação), 'uniforme' (invariante com padrões uniformes de até duas transições), 'nri_uniforme' (Variante de padrão uniforme que é invariante em escala de cinza, mas não invariante em rotação) e 'var' (medidas de variância invariante de rotação do contraste de local);
    * radius: distância entre o pixel central e o vizinho; e
    * points: número de pixels vizinhos.
4. Aplicar o algoritmo LBP.

## **4.1 - Aplicando LBP em uma imagem**

In [ ]:
# Importar imagem convertendo em escala de cinza
image1 = io.imread('./Amazon_Bark/01-Apuleia_leiocarpa/NM_01_R_31_I_8.JPG')
image1 = cv2.resize(image1, (4000, 3000))
imgray = cv2.cvtColor(image1, cv2.COLOR_BGR2GRAY)                                # Converte a imagem para escala de cinza

# Definindo tipo de LBP (P,R = 8,1)
METHOD = 'nri_uniform'    # (uniform)                                            # Padrão uniforme com até duas transições (0-1, 1-0)
radius = 1                                                                       # Distância entre o pixel central e o vizinho
n_points = 8                                                                     # Número de pixel vizinhos

# Aplicando algoritmo LBP a imagem
lbp_img = local_binary_pattern(imgray, n_points, radius, METHOD)
# print(lbp_img.flatten())
#n_bins = int((lbp_img.max() + 1))                                            # Número de bins do histograma
#(hist, bins) = np.histogram(lbp_img.flatten(), bins=n_bins, range=[0, n_bins], density=False)

# Plota as imagens: original, escala de cinza e LBP
plt.rcParams['figure.figsize'] = [15, 6]
plt.subplot(131);plt.imshow(image1)
plt.subplot(132);plt.imshow(imgray, cmap='gray')
plt.subplot(133);plt.imshow(lbp_img, cmap='gray')
plt.show()

# Plota os histogramas de cada imagem
plt.rcParams['figure.figsize'] = [15, 6]
plt.subplot(131);plt.hist(image1.ravel(), bins = int((image1.max()+1)))
plt.subplot(132);plt.hist(imgray.ravel(), bins = 256)
plt.subplot(133);plt.hist(lbp_img.ravel(), bins = 10, range = (0,10))
plt.show()

## **4.2 - Aplicando LBP na base completa**

  * Para aplicar o algoritmo LBP a todas as imagens da base de dados, foi criada uma função 'features_extraction_LBP(P, R, Method) em que os parâmetros LBP são repassados de acordo com a configuração desejada;
  * Neste script mantivemos apenas os nomes das espécies e as amostras as quais as imagens pertecem, porém, as informações sobre caracteristica botânica, área, dispositivo e número da imagem poderá ser acrescentada futuramente, apenas fazendo referência a posição dessa informação no código.

In [ ]:
# ================================================================================
# Função para extração de características com algoritmo LBP
# ================================================================================

def feature_extraction_LBP(P, R, METHOD):                                        # P = pontos, R = raio, METHOD = método
    data = pd.read_excel('./Class.xlsx')                                         # Tabela com os códigos dos nomes das espécies
    imagens = glob.glob('./Amazon_Bark/*/*')                                     # Diretório com imagens

    features = []                                                                # Criando listas para armazenar os dados
    labels = []
    dim = (4000, 3000)

    for filename in imagens:                                                     # Para cada imagem no diretório
        print(os.path.join('.', filename))
        image = io.imread(filename)                                              # carregando imagens
        image = cv2.resize(image, dim)                                           # Redimensiona (4000, 3000)
        grayscale_image = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)                # Converte a imagem para escala de cinza

        lbp = local_binary_pattern(grayscale_image, P, R, METHOD)                # Aplica o algoritmo LBP

        n_bins = int((lbp.max() + 1))                                            # Número de bins do histograma
        (hist, bins) = np.histogram(lbp.flatten(), bins=n_bins, range=[0, n_bins], density=False)
        features.append(hist)                                                    # Adiciona o histograma a lista de features
        #print(features)

        filename_parts = os.path.split(filename)[-1].split('_')                  # Separa o nome do arquivo em partes
        sp = filename_parts[1]                                                   # Código da espécie
        amostra = filename_parts[3]                                              # Número da amostra
        label = (sp, amostra)                                                    # Adiciona a espécie e a amostra a lista de labels
        labels.append(label)                                                     # Adiciona a lista de labels

    df1 = pd.DataFrame(features, columns=[f"V{str(col)}_{METHOD[0:3]}{P}{R}" for col in range(len(features[0]))])    # Gerando o dataframe de features nomeando as colunas
    #scaler = MinMaxScaler()                                                     # Normalizando os dados
    #df1 = pd.DataFrame(scaler.fit_transform(df1), columns=df1.columns)          # Normalizando os dados

    df2 = pd.DataFrame(labels, columns=['Class', 'Sample'])                      # Gerando o dataframe de labels
    df2['Class'] = df2['Class'].astype(int)                                      # Convertendo os códigos das espécies para inteiro

    # Substituição dos nomes das espécies
    species_mapping = data.set_index('n')['Specie'].to_dict()
    df2['Class'] = df2['Class'].map(species_mapping)

    df = pd.concat([df1, df2], axis=1)                                           # Concatenando os dataframes

    path = './Output/Features'

    if not os.path.exists(path):
      os.makedirs(path)
      print("Folder %s created!" % path)
    else:
      print("Folder %s already exists" % path)

    df.to_csv(f'{path}/LBP_{METHOD}_{P}{R}.csv',
              index=False, header=True, sep=',', encoding='utf-8')

    return df

### ***4.3.3 - $LBP^{riu2}_{P,R}$ - LBP UNIFORME E INVARIANTE À ROTAÇÃO***

In [ ]:
# Extraindo características do LBP Uniforme (3 combinações)

#feature_extraction_LBP(P=8, R=1, METHOD='uniform')                               # 10 features
#feature_extraction_LBP(P=16, R=2, METHOD='uniform')                               # 18 features
#feature_extraction_LBP(P=24, R=3, METHOD='uniform')                               # 26 features

### ***4.3.4 - $LBP^{u2}_{P,R}$ - LBP UNIFORME E NÃO INVARIANTE À ROTAÇÃO***

In [ ]:
# Extraindo características do LBP Uniforme e Não Invariante (3 combinações)
#feature_extraction_LBP(P=8, R=1, METHOD='nri_uniform')                            # 59 features
#feature_extraction_LBP(P=16, R=2, METHOD='nri_uniform')                           # 243 features
#feature_extraction_LBP(P=24, R=3, METHOD='nri_uniform')                           # 555 features